### Calculate distance between all offshore wind areas

In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [2]:
df_OW_nodes = pd.read_csv("Output/OW_nodes.csv")
df_OW_nodes["geometry"] = df_OW_nodes["geometry"].apply(wkt.loads)
OW_nodes = gpd.GeoDataFrame(df_OW_nodes, crs="EPSG:4326")
OW_nodes = OW_nodes.to_crs("EPSG:25833")
OW_nodes

,Node,Longitude,Latitude,geometry
0,Moray Firth,-2.990229,58.170702,POINT (-550382.221 6589636.681)
1,Firth of Forth,-2.047535,56.363729,POINT (-547016.494 6377809.984)
2,Dogger Bank,2.333333,54.833333,POINT (-311348.707 6150061.633)
3,Hornsea,1.845196,53.820112,POINT (-363643.253 6044135.822)
4,Outer Dowsing,0.905074,53.311186,POINT (-436309.630 6000047.419)
5,Norfolk,2.168638,52.782831,POINT (-363440.248 5925669.401)
6,East Anglia,1.883496,51.890054,POINT (-400723.386 5830562.425)
7,Borssele,3.197221,51.957807,POINT (-309616.206 5822431.844)
8,Hollandsee Kust,4.008061,52.462704,POINT (-245595.575 5869521.124)
9,Helgoländer Bucht,7.167722,54.322931,POINT (-8909.976 6047781.867)


In [3]:
results = None
for row in OW_nodes.itertuples():
    other_OW_nodes = OW_nodes[OW_nodes.index < row.Index]
    distances = other_OW_nodes.geometry.distance(row.geometry)
    distances_OW_nodes = other_OW_nodes.copy()
    distances_OW_nodes["distance"] = distances / 1000 # m to km
    for name, value in row._asdict().items():
        distances_OW_nodes[f"to_{name}"] = value

    if results is None:
        results = distances_OW_nodes
    else:
        results = pd.concat([results, distances_OW_nodes])

results

,Node,Longitude,Latitude,geometry,distance,to_Index,to_Node,to_Longitude,to_Latitude,to_geometry
0,Moray Firth,-2.990229,58.170702,POINT (-550382.221 6589636.681),211.853435,1,Firth of Forth,-2.047535,56.363729,POINT (-547016.4941446853 6377809.983783474)
0,Moray Firth,-2.990229,58.170702,POINT (-550382.221 6589636.681),500.363112,2,Dogger Bank,2.333333,54.833333,POINT (-311348.7073516828 6150061.633228509)
1,Firth of Forth,-2.047535,56.363729,POINT (-547016.494 6377809.984),327.732539,2,Dogger Bank,2.333333,54.833333,POINT (-311348.7073516828 6150061.633228509)
0,Moray Firth,-2.990229,58.170702,POINT (-550382.221 6589636.681),576.578381,3,Hornsea,1.845196,53.820112,POINT (-363643.25345733296 6044135.821791392)
1,Firth of Forth,-2.047535,56.363729,POINT (-547016.494 6377809.984),380.741634,3,Hornsea,1.845196,53.820112,POINT (-363643.25345733296 6044135.821791392)
...,...,...,...,...,...,...,...,...,...,...
28,Nordavind D,18.808962,71.450117,POINT (635164.986 7931860.745),1869.151340,33,Energyhub Central,3.290000,56.050000,POINT (-227445.80113339517 6273660.105354942)
29,Nordavind C,20.019911,71.728123,POINT (675483.230 7965912.714),1918.071877,33,Energyhub Central,3.290000,56.050000,POINT (-227445.80113339517 6273660.105354942)
30,Vestavind E,3.853030,59.109692,POINT (-136458.754 6605653.720),344.235969,33,Energyhub Central,3.290000,56.050000,POINT (-227445.80113339517 6273660.105354942)
31,Sørvest D,3.840226,56.464644,POINT (-185922.125 6313761.564),57.726447,33,Energyhub Central,3.290000,56.050000,POINT (-227445.80113339517 6273660.105354942)


In [5]:
results = results[["Node", "to_Node", "distance"]]
results

,Node,to_Node,distance
0,Moray Firth,Firth of Forth,211.853435
0,Moray Firth,Dogger Bank,500.363112
1,Firth of Forth,Dogger Bank,327.732539
0,Moray Firth,Hornsea,576.578381
1,Firth of Forth,Hornsea,380.741634
...,...,...,...
28,Nordavind D,Energyhub Central,1869.151340
29,Nordavind C,Energyhub Central,1918.071877
30,Vestavind E,Energyhub Central,344.235969
31,Sørvest D,Energyhub Central,57.726447


Export CSV

In [6]:
results.to_csv("Output/dist_offshore_areas.csv", index=False, encoding='utf-8-sig')